In [ ]:
from IPython.display import Markdown

sql_script = """
--TASK 1: OVERALL ROMI
SELECT
  (SUM(Revenue) - SUM(Marketing_Spent)) / SUM(Marketing_Spent) * 100 AS overall_ROMI
FROM Marketing_01;

--TASK 2: ROMI BY CAMPAIGN
SELECT Campaign_Name,
  SUM(Marketing_Spent) AS Spent,
  (SUM(Revenue) - SUM(Marketing_Spent)) * 100.0 / SUM(Marketing_Spent) AS ROMI_By_Campaign,
  (SUM(Clicks) * 100.0) / NULLIF(SUM(Impressions), 0) AS CTR,
  SUM(Leads) * 100.0 / NULLIF(SUM(Clicks), 0) AS CR1,
  SUM(Orders) * 100.0 / NULLIF(SUM(Leads), 0) AS CR2,
  SUM(Revenue) / NULLIF(SUM(Orders), 0) AS AOV,
  SUM(Marketing_Spent) / NULLIF(SUM(Clicks), 0) AS CPC,
  SUM(Marketing_Spent) / NULLIF(SUM(Leads), 0) AS CPL,
  SUM(Marketing_Spent) / NULLIF(SUM(Orders), 0) AS CAC,
  SUM(Revenue) - SUM(Marketing_Spent) AS Gross_profit
FROM Marketing_01
GROUP BY Campaign_Name;

--TASK 3: PERFORMANCE OF THE CAMPAIGN DEPENDING ON THE DATE
SELECT Date, Marketing_Spent
FROM Marketing_01
ORDER BY Marketing_Spent DESC
LIMIT 1;

WITH CR AS (
SELECT Date, Revenue
,Leads*100.0 /NULLIF(Clicks,0)  as CR1
, Orders * 100.0 / NULLIF(Leads,0) as CR2
FROM Marketing_01
)

--CR 1
SELECT Date, Revenue, CR1
FROM CR
WHERE CR1 IS NOT NULL
ORDER BY CR1 DESC, Revenue DESC
--ORDER BY CR1 ASC, Revenue DESC
LIMIT 10;

--CR 2
WITH CR AS (
SELECT Date, Revenue
,Leads*100.0 /NULLIF(Clicks,0)  as CR1
, Orders * 100.0 / NULLIF(Leads,0) as CR2
FROM Marketing_01
)
SELECT Date, Revenue, CR2
FROM CR
WHERE CR2 IS NOT NULL
ORDER BY CR2 DESC, Revenue DESC
--ORDER BY CR2 ASC, Revenue DESC
LIMIT 10;

SELECT  Date
, SUM (Revenue)/NULLIF( SUM (Orders),0) as AOV
FROM Marketing_01
GROUP BY 1
ORDER BY SUM (Revenue)/NULLIF( SUM (Orders),0) DESC;


--TASK 4: WHEN BUYERS ARE MORE ACTIVE? WHAT IS THE AVERAGE REVENUE ON WEEKDAYS AND WEEKENDS?

WITH c_day_of_week AS (
SELECT*
,strftime ('%w', Date) as day_type
, CASE
WHEN strftime ('%w', Date) = '1' THEN 'Monday'     --0:Sun, 6:Sat
WHEN strftime ('%w', Date) = '2' THEN 'Tuesday'
WHEN strftime ('%w', Date) = '3' THEN 'Wednesday'
WHEN strftime ('%w', Date) = '4' THEN 'Thursday'
WHEN strftime ('%w', Date) = '5' THEN 'Friday'
ELSE 'weekend'
END AS day_of_week
FROM Marketing_01
)
SELECT day_of_week
,SUM (Clicks) as Clicks
, SUM (Impressions) as Impressions
, SUM (Orders) as Orders_amount
, SUM (Revenue) as Total_Revenue
,SUM (Clicks)*100.0/ NULLIF(SUM(Impressions),0) as CTR
, SUM(Revenue)/NULLIF(SUM(Orders),0) AS AOV
FROM c_day_of_week
GROUP BY day_of_week
ORDER BY  CASE day_of_week
WHEN 'Monday' THEN 1
WHEN 'Tuesday' THEN 2
WHEN   'Wednesday' THEN 3
WHEN   'Thursday' THEN 4
WHEN  'Friday' THEN 5
ELSE 6
END ASC;
---ORDER BY SUM(Orders) DESC;

--TASK 5: WHICH TYPES OF CAMPAIGNS WORK BEST - SOCIAL, BANNER, INFLUENCER, OR A SEARCH?

WITH Total AS (
SELECT
SUM (Revenue) as Overall_Revenue
FROM Marketing_01
)
SELECT Category
, SUM (Revenue)*100.0/NULLIF(Total.Overall_Revenue,0 ) as Revenue_by_Share
,(SUM(Revenue) - SUM (Marketing_Spent))*100.0/ NULLIF(SUM(Marketing_Spent),0)  AS ROMI
, (SUM(Clicks)*100.0)/ NULLIF(SUM (Impressions),0) as CTR
, SUM (Leads)*100.0 /NULLIF( SUM (Clicks) , 0)AS CR1
, SUM (Orders)*100.0 / NULLIF(SUM (Leads),0) AS CR2
, SUM(Revenue)/NULLIF(SUM (Orders),0) as AOV
, SUM (Marketing_Spent)/NULLIF( SUM (Clicks),0) AS CPC
, SUM (Marketing_Spent)/ NULLIF(SUM (Leads),0) AS CPL
, SUM (Marketing_Spent)/ NULLIF(SUM (Orders),0) AS CAC
FROM Marketing_01
CROSS JOIN Total
GROUP BY 1;

--TASK 6: WHICH GEO LOCATIONS ARE BETTER FOR TARGETING - TIER 1 OR TIER 2 CITIES?

WITH total AS (
SELECT
SUM (Revenue) as Overall_Revenue
FROM Marketing_01
)
SELECT
CASE
WHEN Campaign_Name LIKE '%tier1%' THEN 'Tier 1'
WHEN Campaign_Name LIKE '%tier2%' THEN 'Tier 2'
ELSE 'others'
END AS City_Tier
, SUM(Revenue)*100.0/ NULLIF(SUM(Total.Overall_Revenue),0) as Revenue_Share
,(SUM(Revenue) - SUM (Marketing_Spent))*100.0/ NULLIF(SUM(Marketing_Spent),0)  AS ROMI
, (SUM(Clicks)*100.0)/ NULLIF(SUM (Impressions),0) as CTR
, SUM (Leads)*100.0 /NULLIF( SUM (Clicks) , 0)AS CR1
, SUM (Orders)*100.0 / NULLIF(SUM (Leads),0) AS CR2
, SUM(Revenue)/NULLIF(SUM (Orders),0) as AOV
, SUM (Marketing_Spent)/ NULLIF(SUM (Clicks),0) AS CPC
, SUM (Marketing_Spent)/ NULLIF(SUM (Leads),0) AS CPL
, SUM (Marketing_Spent)/ NULLIF(SUM (Orders),0) AS CAC
, SUM (Revenue) - SUM (Marketing_Spent) AS Gross_profit
FROM Marketing_01
GROSS JOIN Total
GROUP BY CASE
WHEN Campaign_Name LIKE '%tier1%' THEN 'Tier 1'
WHEN Campaign_Name LIKE '%tier2%' THEN 'Tier 2'
ELSE 'others'
END;

-- PERFORMANCE OVER TIME
SELECT Date
,SUM (Marketing_Spent) as Marketing_Spent
,SUM (Revenue) as Revenue
,SUM (Impressions) as Impressions
,SUM(Clicks) as Clicks
,SUM (Leads) as Leads
,SUM (Orders) as Orders
-- Metrics
, SUM(Marketing_Spent)/NULLIF(SUM(Clicks),0) as CPC
, SUM (Marketing_Spent)/ NULLIF(SUM(Leads),0) as CPL
, SUM (Marketing_Spent)/NULLIF(SUM(Orders),0) as CAC
, SUM(Clicks)*100.0/NULLIF(SUM(Impressions),0) as CTR
,SUM(Revenue)/NULLIF(SUM(Orders),0) as AOV
, SUM(Revenue)-SUM(Marketing_Spent) as Gross_Profit
FROM Marketing_01
GROUP BY Date;
"""
display(Markdown(f"```sql\n{sql_script}\n```"))

```sql

--TASK 1: OVERALL ROMI
SELECT
  (SUM(Revenue) - SUM(Marketing_Spent)) / SUM(Marketing_Spent) * 100 AS overall_ROMI 
FROM Marketing_01;

--TASK 2: ROMI BY CAMPAIGN
SELECT Campaign_Name,
  SUM(Marketing_Spent) AS Spent,
  (SUM(Revenue) - SUM(Marketing_Spent)) * 100.0 / SUM(Marketing_Spent) AS ROMI_By_Campaign,
  (SUM(Clicks) * 100.0) / NULLIF(SUM(Impressions), 0) AS CTR,
  SUM(Leads) * 100.0 / NULLIF(SUM(Clicks), 0) AS CR1,
  SUM(Orders) * 100.0 / NULLIF(SUM(Leads), 0) AS CR2,
  SUM(Revenue) / NULLIF(SUM(Orders), 0) AS AOV,
  SUM(Marketing_Spent) / NULLIF(SUM(Clicks), 0) AS CPC,
  SUM(Marketing_Spent) / NULLIF(SUM(Leads), 0) AS CPL,
  SUM(Marketing_Spent) / NULLIF(SUM(Orders), 0) AS CAC,
  SUM(Revenue) - SUM(Marketing_Spent) AS Gross_profit
FROM Marketing_01 
GROUP BY Campaign_Name;

--TASK 3: PERFORMANCE OF THE CAMPAIGN DEPENDING ON THE DATE
SELECT Date, Marketing_Spent
FROM Marketing_01 
ORDER BY Marketing_Spent DESC 
LIMIT 1;

WITH CR AS (
SELECT Date, Revenue
,Leads*100.0 /NULLIF(Clicks,0)  as CR1
, Orders * 100.0 / NULLIF(Leads,0) as CR2 
FROM Marketing_01 
) 

--CR 1  
SELECT Date, Revenue, CR1 
FROM CR  
WHERE CR1 IS NOT NULL 
ORDER BY CR1 DESC, Revenue DESC 
--ORDER BY CR1 ASC, Revenue DESC  
LIMIT 10;

--CR 2 
WITH CR AS (
SELECT Date, Revenue
,Leads*100.0 /NULLIF(Clicks,0)  as CR1
, Orders * 100.0 / NULLIF(Leads,0) as CR2 
FROM Marketing_01 
) 
SELECT Date, Revenue, CR2 
FROM CR 
WHERE CR2 IS NOT NULL 
ORDER BY CR2 DESC, Revenue DESC  
--ORDER BY CR2 ASC, Revenue DESC 
LIMIT 10;

SELECT  Date
, SUM (Revenue)/NULLIF( SUM (Orders),0) as AOV 
FROM Marketing_01 
GROUP BY 1
ORDER BY SUM (Revenue)/NULLIF( SUM (Orders),0) DESC; 


--TASK 4: WHEN BUYERS ARE MORE ACTIVE? WHAT IS THE AVERAGE REVENUE ON WEEKDAYS AND WEEKENDS?

WITH c_day_of_week AS (
SELECT*
,strftime ('%w', Date) as day_type 
, CASE        
WHEN strftime ('%w', Date) = '1' THEN 'Monday'     --0:Sun, 6:Sat 
WHEN strftime ('%w', Date) = '2' THEN 'Tuesday'
WHEN strftime ('%w', Date) = '3' THEN 'Wednesday'
WHEN strftime ('%w', Date) = '4' THEN 'Thursday' 
WHEN strftime ('%w', Date) = '5' THEN 'Friday' 
ELSE 'weekend'
END AS day_of_week
FROM Marketing_01 
) 
SELECT day_of_week
,SUM (Clicks) as Clicks 
, SUM (Impressions) as Impressions 
, SUM (Orders) as Orders_amount 
, SUM (Revenue) as Total_Revenue
,SUM (Clicks)*100.0/ NULLIF(SUM(Impressions),0) as CTR 
, SUM(Revenue)/NULLIF(SUM(Orders),0) AS AOV
FROM c_day_of_week 
GROUP BY day_of_week
ORDER BY  CASE day_of_week
WHEN 'Monday' THEN 1
WHEN 'Tuesday' THEN 2
WHEN   'Wednesday' THEN 3
WHEN   'Thursday' THEN 4
WHEN  'Friday' THEN 5
ELSE 6
END ASC; 
---ORDER BY SUM(Orders) DESC;

--TASK 5: WHICH TYPES OF CAMPAIGNS WORK BEST - SOCIAL, BANNER, INFLUENCER, OR A SEARCH?

WITH Total AS (
SELECT 
SUM (Revenue) as Overall_Revenue
FROM Marketing_01
) 
SELECT Category
, SUM (Revenue)*100.0/NULLIF(Total.Overall_Revenue,0 ) as Revenue_by_Share
,(SUM(Revenue) - SUM (Marketing_Spent))*100.0/ NULLIF(SUM(Marketing_Spent),0)  AS ROMI
, (SUM(Clicks)*100.0)/ NULLIF(SUM (Impressions),0) as CTR
, SUM (Leads)*100.0 /NULLIF( SUM (Clicks) , 0)AS CR1
, SUM (Orders)*100.0 / NULLIF(SUM (Leads),0) AS CR2
, SUM(Revenue)/NULLIF(SUM (Orders),0) as AOV
, SUM (Marketing_Spent)/NULLIF( SUM (Clicks),0) AS CPC
, SUM (Marketing_Spent)/ NULLIF(SUM (Leads),0) AS CPL
, SUM (Marketing_Spent)/ NULLIF(SUM (Orders),0) AS CAC
FROM Marketing_01 
CROSS JOIN Total 
GROUP BY 1; 

--TASK 6: WHICH GEO LOCATIONS ARE BETTER FOR TARGETING - TIER 1 OR TIER 2 CITIES? 

WITH total AS (
SELECT 
SUM (Revenue) as Overall_Revenue 
FROM Marketing_01 
)
SELECT 
CASE 
WHEN Campaign_Name LIKE '%tier1%' THEN 'Tier 1' 
WHEN Campaign_Name LIKE '%tier2%' THEN 'Tier 2' 
ELSE 'others'
END AS City_Tier 
, SUM(Revenue)*100.0/ NULLIF(SUM(Total.Overall_Revenue),0) as Revenue_Share 
,(SUM(Revenue) - SUM (Marketing_Spent))*100.0/ NULLIF(SUM(Marketing_Spent),0)  AS ROMI
, (SUM(Clicks)*100.0)/ NULLIF(SUM (Impressions),0) as CTR
, SUM (Leads)*100.0 /NULLIF( SUM (Clicks) , 0)AS CR1
, SUM (Orders)*100.0 / NULLIF(SUM (Leads),0) AS CR2 
, SUM(Revenue)/NULLIF(SUM (Orders),0) as AOV
, SUM (Marketing_Spent)/ NULLIF(SUM (Clicks),0) AS CPC  
, SUM (Marketing_Spent)/ NULLIF(SUM (Leads),0) AS CPL  
, SUM (Marketing_Spent)/ NULLIF(SUM (Orders),0) AS CAC  
, SUM (Revenue) - SUM (Marketing_Spent) AS Gross_profit 
FROM Marketing_01
GROSS JOIN Total  
GROUP BY CASE 
WHEN Campaign_Name LIKE '%tier1%' THEN 'Tier 1' 
WHEN Campaign_Name LIKE '%tier2%' THEN 'Tier 2' 
ELSE 'others'
END;

-- PERFORMANCE OVER TIME 
SELECT Date
,SUM (Marketing_Spent) as Marketing_Spent 
,SUM (Revenue) as Revenue 
,SUM (Impressions) as Impressions
,SUM(Clicks) as Clicks 
,SUM (Leads) as Leads 
,SUM (Orders) as Orders 
-- Metrics
, SUM(Marketing_Spent)/NULLIF(SUM(Clicks),0) as CPC 
, SUM (Marketing_Spent)/ NULLIF(SUM(Leads),0) as CPL 
, SUM (Marketing_Spent)/NULLIF(SUM(Orders),0) as CAC 
, SUM(Clicks)*100.0/NULLIF(SUM(Impressions),0) as CTR 
,SUM(Revenue)/NULLIF(SUM(Orders),0) as AOV 
, SUM(Revenue)-SUM(Marketing_Spent) as Gross_Profit 
FROM Marketing_01 
GROUP BY Date; 

```